<a href="https://colab.research.google.com/github/amannizamani/LangChain-RAG-Project/blob/main/RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -q -U python-dotenv  langchain pinecone-client google-generativeai openai tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.6 MB/s eta 0:00:00


In [ ]:
import os
from dotenv import load_dotenv
from google.colab import userdata
# Retrieve and set the environment variables
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
os.environ['PINECONE_ENVIRONMENT'] = userdata.get('PINECONE_ENVIRONMENT')
os.environ['GOOGLE_API_KEY_1'] = userdata.get('GOOGLE_API_KEY_1')

In [ ]:
from pinecone import Pinecone
pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'], environment=os.environ['PINECONE_ENVIRONMENT'])

In [ ]:
index_name = "gemini-rag-index" # or your index name

index = pc.Index(index_name)
print(f"connected to index: {index_name}")
        # do something


connected to index: gemini-rag-index


In [ ]:
%pip install -q -U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.1 MB/s eta 0:00:00


In [ ]:
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=os.environ["GOOGLE_API_KEY_1"]
    )

In [ ]:
%pip install -q -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.0 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


# Load documents
loader = TextLoader("/document.txt")  # Replace with your file
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [ ]:
from tqdm import tqdm

# Create embeddings and upload to Pinecone
for doc in tqdm(docs):
    vector = embeddings.embed_query(doc.page_content)
    index.upsert([(doc.metadata["source"], vector ,{'text':doc.page_content})])

100%|██████████| 4/4 [00:02<00:00,  1.98it/s]


In [ ]:
from langchain.vectorstores import Pinecone

retriever =  Pinecone.from_existing_index(index_name=index_name, embedding=embeddings, text_key="text")

In [ ]:
%pip install -q -U langchain-google-genai
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_model =  ChatGoogleGenerativeAI(api_key=os.environ.get("GOOGLE_API_KEY_1"),model="gemini-1.5-flash", temperature=0.7)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.vectorstores.base import VectorStoreRetriever

retriever = VectorStoreRetriever(vectorstore=Pinecone.from_existing_index(index_name=index_name, embedding=embeddings, text_key="text"))

qa_chain = RetrievalQA.from_chain_type(
    llm=gemini_model,
    chain_type="stuff",  # Other options: "map_reduce", "refine"
    retriever=retriever
)

In [ ]:
query = "what happens when you integrate the AI in genetics"
response = qa_chain.run(query)
print(f"Laiba's Question: {query}")
print()
print(f"LLM's Response: {response}")

Laiba's Question: what happens when you integrate the AI in genetics

LLM's Response: I don't know.

